In [ ]:
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas
from functools import partial
import warnings
import numpy as np
from concurrent.futures import ProcessPoolExecutor
warnings.filterwarnings('ignore')
import grpc
import time

In [10]:
key_path = '/home/jupyter/d00_key.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

QUERY = """
    SELECT * FROM `dw-bq-data-d00.SANDBOX_ANALYTICS.layer2_Midwest0`
    """
trans_hist = bq_client.query(QUERY).result().to_dataframe(bqstorage_client=bqstorageclient)
trans_hist.head()
QUERY = """
    SELECT * FROM `dw-bq-data-d00.SANDBOX_ANALYTICS.prod_type_midwest`
    """
prod_type_recs = bq_client.query(QUERY).result().to_dataframe(bqstorage_client=bqstorageclient)
prod_type_recs.head()

,focus_pdm_prod_type_id,rn,recomm_pdm_prod_type_id
0,2,0,2
1,2,3,1576
2,2,4,1490
3,2,6,1917
4,2,8,2776


In [5]:
trans_hist.head()

,customer_id,pdm_prod_type_id,trans_count
0,1000231335060,2870,6
1,1000276844873,2911,5
2,1000060304671,1490,5
3,1000104719114,540,5
4,1000106793996,1144,9


In [11]:
prod_type=prod_type_recs["focus_pdm_prod_type_id"].unique() 
sku=trans_hist[["item_sku_num","pdm_prod_type_id"]].drop_duplicates()

In [ ]:

# output0 = pandas.DataFrame()
# output1 = pandas.DataFrame()
# for i in prod_type[0:40]:
#     prod = i
#     print("Product Type:",prod)
#     rec_set = prod_type_recs[prod_type_recs['focus_pdm_prod_type_id'] ==
#                     i]['recomm_pdm_prod_type_id']
#     rec_trans = trans_hist[trans_hist['pdm_prod_type_id'].isin(rec_set)]
#     trans = rec_trans[rec_trans['pdm_prod_type_id'] ==
#                        i]['transaction_guid'].unique()
#     print("Number of transactions:",len(trans))
#     focus_rec_trans_set = rec_trans[rec_trans['transaction_guid'].isin(trans)][[
#             'transaction_guid', 'item_sku_num'
#         ]]
#     vc = focus_rec_trans_set['transaction_guid'].value_counts()
#     df = focus_rec_trans_set[focus_rec_trans_set['transaction_guid'].isin(vc[vc >= 2].index)]
#     def trans_loops(trans,prod=i,df=df):
#         output = pandas.DataFrame()
#         for i in trans:
#             iden = i
#             new_list = df[df['transaction_guid'] == i][['item_sku_num']]
#             new_list = new_list.item_sku_num.tolist()
#             output1 = pandas.DataFrame()
#             o = []
#             o1 = []
#             o3 = []
#             o4 = []
#             for i in range(0, len(new_list)):
#                 for j in range(0, len(new_list)):

#                     # checking if i and j indexes are not on same element
#                     if (i != j) and new_list[i] in sku[
#                             sku['pdm_prod_type_id'] == prod][[
#                                     'item_sku_num'
#                                 ]].item_sku_num.to_list() and new_list[j] in sku[
#                                     sku['pdm_prod_type_id'] != prod][[
#                                         'item_sku_num'
#                                     ]].item_sku_num.to_list():
#                             # add to output
#                         o.append(new_list[i])
#                         o1.append(new_list[j])
#                         o3.append(iden)

#             output1['Trans'] = o3
#             output1['Focus'] = o
#             output1['Recomm'] = o1

#             output = pandas.concat([output, output1])
#             return output
        
#     trans_splits = np.array_split(trans, 70)
#     with mp.Pool(70) as pool:
#         int_result = pandas.concat(pool.map(trans_loops, trans_splits))
#     output0 = pandas.concat([output0, int_result])
# output1 = pandas.concat([output1, output0])
# result = output1.groupby(['Focus', 'Recomm']).count()    

In [ ]:
output1

In [12]:
def trans_loops(trans, prod, df):
    output = pandas.DataFrame()
    for i in trans:
        iden = i
        new_list = df[df['customer_id'] == i][['item_sku_num']]
        new_list = new_list.item_sku_num.tolist()
        output1 = pandas.DataFrame()
        o = []
        o1 = []
        o3 = []
        o4 = []
        for i in range(0, len(new_list)):
            for j in range(0, len(new_list)):

                # checking if i and j indexes are not on same element
                if (i!=j) and new_list[i] in sku[sku['pdm_prod_type_id']==prod][['item_sku_num']].item_sku_num.to_list() and new_list[j] in sku[sku['pdm_prod_type_id']!=prod][['item_sku_num']].item_sku_num.to_list():
                    o.append(new_list[i])
                    o1.append(new_list[j])
                    o3.append(iden)

        output1['customer_id'] = o3
        output1['Focus'] = o
        output1['Recomm'] = o1

        output = pandas.concat([output, output1])
    return output

In [14]:
output0 = pandas.DataFrame()
output1 = pandas.DataFrame()
tic = time.time()
print("Start time:",tic)
for i in prod_type:
    prod = i
    print("Product Type:",prod)
    tic = time.time()
    print("Start time:",tic)
    rec_set = prod_type_recs[prod_type_recs['focus_pdm_prod_type_id'] ==
                    i]['recomm_pdm_prod_type_id']
    rec_trans = trans_hist[trans_hist['pdm_prod_type_id'].isin(rec_set)]
    trans = rec_trans[rec_trans['pdm_prod_type_id'] ==
                       i]['customer_id'].unique()
    print("Number of transactions:",len(trans))
    focus_rec_trans_set = rec_trans[rec_trans['customer_id'].isin(trans)][[
            'customer_id', 'item_sku_num'
        ]]
    vc = focus_rec_trans_set['customer_id'].value_counts()
    df = focus_rec_trans_set[focus_rec_trans_set['customer_id'].isin(vc[vc >= 2].index)]
    
    trans_splits = np.array_split(trans, 72)
    with grpc.insecure_channel('localhost:50051') as channel:
        with ProcessPoolExecutor(72) as pool:
            int_result = pandas.concat(pool.map(partial(trans_loops, prod=i, df=df), trans_splits))
            output0 = pandas.concat([output0, int_result])
            output0.to_pickle("./output0.pkl") 
            toc = time.time()
            print("End time:",toc)
            print('Done in {:.4f} seconds'.format(toc-tic))
output1 = pandas.concat([output1, output0])
output1.to_pickle("./output1.pkl") 
result = output1.groupby(['Focus', 'Recomm']).count()    
toc = time.time()
print("End time:",toc)
print('Done in {:.4f} seconds'.format(toc-tic))

Start time: 1653341626.3034856
Product Type: 2
Start time: 1653341626.3038428
Number of transactions: 4568
End time: 1653341654.0477486
Done in 27.7439 seconds
Product Type: 3
Start time: 1653341654.1086643
Number of transactions: 116
End time: 1653341656.7461095
Done in 2.6374 seconds
Product Type: 4
Start time: 1653341656.8027534
Number of transactions: 4
End time: 1653341658.9939332
Done in 2.1912 seconds
Product Type: 5
Start time: 1653341659.0353072
Number of transactions: 14524
End time: 1653341680.6873405
Done in 21.6520 seconds
Product Type: 6
Start time: 1653341680.7509458
Number of transactions: 4
End time: 1653341682.940002
Done in 2.1891 seconds
Product Type: 8
Start time: 1653341682.9816582
Number of transactions: 4118
End time: 1653341703.036432
Done in 20.0548 seconds
Product Type: 9
Start time: 1653341703.102393
Number of transactions: 2080
End time: 1653341708.208377
Done in 5.1060 seconds
Product Type: 10
Start time: 1653341708.2801256
Number of transactions: 7543
End

In [15]:
output0

,customer_id,Focus,Recomm
0,1.001271e+12,69699027.0,69634596.0
1,1.001271e+12,69699027.0,69741367.0
2,1.001271e+12,69699027.0,69734149.0
3,1.001271e+12,69699027.0,69697442.0
0,1.000051e+12,69699026.0,69084751.0
...,...,...,...
0,1.000095e+12,69860140.0,69732919.0
0,1.000683e+12,69860140.0,41434215.0
1,1.000683e+12,69860140.0,69634597.0
2,1.000683e+12,69860140.0,14644490.0


In [16]:
result

customer_id
Focus      Recomm                 
4950.0     14644580.0            1
           14752510.0            2
           15059591.0            1
           15059648.0            2
           15059788.0            1
...                            ...
77604841.0 69661734.0            1
77604860.0 69686622.0            1
77604864.0 69598983.0            1
           69635140.0            1
           69635146.0            1

[6557988 rows x 1 columns]